In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
ostro_dir = 'Osteosarcoma-UT'



# Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [14]:
from sklearn.model_selection import train_test_split
import os

# Define the root directory containing subfolders for each class
root_dir = 'Osteosarcoma-UT'

# Get the list of class names
classes = os.listdir(root_dir)
print(classes)

# Split the data into training and validation sets
train_data = []
val_data = []
for cls in classes:
    cls_dir = os.path.join(root_dir, cls)
    images = [os.path.join(cls_dir, img) for img in os.listdir(cls_dir)]
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    train_data.extend([(img, cls) for img in train_images])
    val_data.extend([(img, cls) for img in val_images])

# Further split the training data into training and testing sets
train_images, test_images = train_test_split(train_data, test_size=0.2, random_state=42)

# Now you have train_images, val_data, and test_images containing paths to images for each set



['Viable', 'Non-Tumor', 'Non-Viable-Tumor']


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame with two columns: 'file_path' and 'label'
train_df = pd.DataFrame(train_data, columns=['file_path', 'label'])
val_df = pd.DataFrame(val_data, columns=['file_path', 'label'])
test_df = pd.DataFrame(test_images, columns=['file_path', 'label'])

# Split the data into file paths and labels
train_data = train_df['file_path']
train_labels = train_df['label']

val_data = val_df['file_path']
val_labels = val_df['label']

test_data = test_df['file_path']
test_labels = test_df['label']

# You can create separate data generators for each set
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 914 validated image filenames belonging to 3 classes.
Found 230 validated image filenames belonging to 3 classes.
Found 183 validated image filenames belonging to 3 classes.


In [16]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [18]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
def create_cnn(input_shape=(224, 224, 3), num_classes=3):  # Assuming 3 classes: non tumor, non viable tumor, viable tumor
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Create CNN model
model = create_cnn()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Change loss function accordingly
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch= len(train_generator),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=[early_stopping])




Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 111, 111, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_28 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_29 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 26, 26, 128)      

In [19]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

5/5 [==============================] - 1s 242ms/step - loss: 0.5593 - accuracy: 0.8562
Test accuracy: 0.856249988079071
